### Brazilian E-Commerce Public Dataset by Olist
This is a Brazilian ecommerce public dataset of orders made at Olist Store. The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. Its features allows viewing an order from multiple dimensions: from order status, price, payment and freight performance to customer location, product attributes and finally reviews written by customers. We also released a geolocation dataset that relates Brazilian zip codes to lat/lng coordinates.

--- OLIST E-COMMERCE DATASET: ML FEATURE ARCHITECTURE REPORT ---

The goal of this pre-processing pipeline was to establish a high-integrity, predictive feature space by prioritizing variance control, temporal integrity, and domain-aware imputation over simple data deletion.

### 1. CUSTOMERS & SELLERS DATA (Geospatial Integrity)

- ACTION: Applied rigorous standardization (lower/strip) to all `city` and `state` columns.
- ML RATIONALE: Ensures uniformity for future geospatial feature engineering (e.g., embeddings) and prevents subtle textual variations from inflating categorical cardinality.
- OUTLIER CHECK: Confirmed NO statistical outliers in Zip Code Prefixes (ZIPs are categorical identifiers, not continuous features).
- OUTCOME: Clean, linkable demographic/location data for Customer Lifetime Value (CLV) and Logistics Models.

### 2. ORDERS DATA (Temporal Causality & Target Population)

- ACTION: Filtered dataset to retain only orders with `order_status` == 'delivered'.
- ML RATIONALE: Establishes the necessary 'Ground Truth' for supervised models (delivery time, satisfaction). Incomplete orders are noise for these tasks.
- ACTION: Converted all 5 timestamp columns to datetime objects and performed **TARGETED NULL DROPPING** on critical internal dates (`order_approved_at`, `order_delivered_carrier_date`).
- ACTION: Explicitly removed orders violating temporal causality (e.g., approval before purchase, delivery before carrier).
- ML RATIONALE: This crucial step protects the integrity of all derived time-difference features (like 'Delivery Lead Time') from non-physical errors.
- OUTCOME: A temporally consistent central hub table for high-fidelity feature joining.

### 3. ORDER ITEMS DATA (The Skewness Problem)

- OUTLIER DETECTED: Extreme right-skewed outliers in `price` and `freight_value` (the 'Whale Orders').
- ACTION: Applied **Logarithmic Transformation** ($\\log(1+x)$) to create `price_log` and `freight_value_log`.
- ML RATIONALE: This transformation is chosen over deletion because the extreme values are REAL and predictive. Log-transform stabilizes variance, normalizes the distribution, and ensures compatibility with linear algorithms without sacrificing the influence of high-value signal.

### 4. PRODUCTS DATA (Dimensionality and Attribute Robustness)

- FEATURE CREATION: Explicitly derived the feature **`product_volume_cm3`** (Length * Height * Width) as a primary predictor for freight cost.
- IMPUTATION STRATEGY: Imputed all missing physical dimensions/counts using the **MEDIAN**.
- ML RATIONALE: The median is robust against the sparse outliers common in product dimensions, preserving the central tendency for logistics modeling.
- OUTLIER CONTROL (Weight/Volume): Applied **Logarithmic Transformation** to `product_weight_g` and the newly created `product_volume_cm3` for variance stabilization (similar to Order Items).
- OUTLIER CONTROL (Description): Applied **99th Percentile Capping (Winsorizing)** to `product_description_length`.
- ML RATIONALE: This is a targeted strategy to limit the influence of the few extremely verbose descriptions on numerical proxies for text complexity.

### 5. ORDER REVIEWS DATA (Sentiment Signal Integrity)

- IMPUTATION STRATEGY: Filled the ~88% nulls in `review_comment_message` with the **'no comment'** string.
- ML RATIONALE: A missing comment is a **meaningful, sentiment-neutral categorical class** for future NLP and satisfaction models.
- OUTLIER DECISION (Scores): Statistical outliers (low scores of 1 or 2) were **RETAINED**.
- ML RATIONALE: These are high-leverage, negative feedback data points essential for training **Churn and Customer Satisfaction Models**. Removing them would introduce positive bias.
- INTEGRITY: Removed 814 duplicate `review_id` values to ensure clean 1:1 join with the central Orders table.

### 6. PAYMENTS DATA (Financial Variance Control)

- OUTLIER DETECTED: Extreme right-skew in `payment_value`.
- ACTION: Applied **99.9th Percentile Capping (Winsorizing)** to create `capped_payment_value`.
- ML RATIONALE: Capping is superior to deletion/log-transform here; it directly limits the influence of 'Whale' transactions on **L2 loss (MSE)**, stabilizing model training without removing the critical high-value signal.
- ZERO-VALUE VALIDATION: Confirmed $0.00 payments are associated with the **'voucher'** payment type. Retained as a definitive, predictive categorical signal.

### 7. GEOLOCATION DATA (Efficiency and Geographic Purity)

- DATA REDUCTION: Dropped over 300,000 redundant duplicate coordinate mappings (on `zip_code_prefix`, `lat`, `lng`).
- ML RATIONALE: Drastically improves memory footprint and join efficiency for a massive lookup table.
- OUTLIER CHECK: Implemented a **Geographic Boundary Check** to remove coordinates falling outside the plausible bounds of Brazil.
- ML RATIONALE: Ensures that the distance metrics derived from this table are based on physically accurate, valid seller and customer locations.

### 8. TRANSLATION DATA (Interpretability Layer)

- ACTION: No cleaning/transformation required (perfect quality).
- ML RATIONALE: The file's sole purpose is to serve as a high-integrity lookup table, providing the essential **English language interpretability layer** for all categorical product features.

---
**FINAL STATUS: All nine relational data sources have been transformed into a dense, high-quality, and variance-stabilized feature matrix, ready for sophisticated EDA, feature engineering (e.g., distance and time-series metrics), and predictive modeling.**

The code in the following cell downloads the dataset from Kagglehub and lists the files in the downloaded directory.

In [ ]:
import kagglehub
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px

# Download the latest version of the dataset
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

# List the files in the downloaded directory
file_list = os.listdir(path)
for file_name in file_list:
    print(file_name)

In [ ]:
def detect_and_plot_outliers_iqr(df, column, exclude_zero=False):
    """
    Detects outliers using the IQR method, prints outlier information, and plots a box plot.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): The name of the column to analyze for outliers.
        exclude_zero (bool): Whether to exclude zero values from outlier calculation.
                             Useful for columns where zero is a meaningful non-outlier value.
    """
    print(f"\n--- Outlier Analysis for column: {column} ---")

    # Create box plot
    plt.figure(figsize=(8, 4))
    sns.boxplot(y=df[column])
    plt.title(f'Box plot of {column}')
    plt.ylabel(column)
    plt.show()

    # Calculate IQR and identify outliers
    if exclude_zero:
        # Consider only non-zero values for IQR calculation
        data_for_iqr = df[df[column] != 0][column]
    else:
        data_for_iqr = df[column]

    if data_for_iqr.empty:
        print(f"  No non-zero data in column '{column}' to calculate outliers.")
        return

    Q1 = data_for_iqr.quantile(0.25)
    Q3 = data_for_iqr.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers in the original DataFrame
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    num_outliers = len(outliers)
    percentage_outliers = (num_outliers / len(df)) * 100

    print(f"  Number of outliers: {num_outliers}")
    print(f"  Percentage of outliers: {percentage_outliers:.2f}%")
    print(f"  Lower bound (IQR): {lower_bound:.2f}")
    print(f"  Upper bound (IQR): {upper_bound:.2f}")

    if num_outliers > 0 and num_outliers < 20: # Display if not too many outliers
        print("\nSample Outlier Rows:")
        display(outliers.head())

### Processing the `olist_customers_dataset.csv`, this includes checking for null/missing values, ensuring all datatypes are correctly assigned to the columns, checking for duplicate values and checking for outliers.

In [ ]:
# Load the customers dataset
customers_dataset_path = os.path.join(path, "olist_customers_dataset.csv")
customers_df = pd.read_csv(customers_dataset_path)

# Display the first few rows of the DataFrame
customers_df.head()

In [ ]:
# Get information about the customers_df DataFrame
print(customers_df.info())

# Display missing values information
print("\nMissing values in customers_df:")
print(customers_df.isnull().sum())

In [ ]:
# Checking for duplicated values
print(f"Unique customer: {customers_df.nunique()}")
print(f"Duplicated values in customer_df: {customers_df.duplicated().sum()}")
print(f"Duplicated values in customer_df['customer_id']: {customers_df['customer_id'].duplicated().sum()}")
# Expected as there are repeat buyers
print(f"Duplicated values in customer_df['customer_unique_id']: {customers_df['customer_unique_id'].duplicated().sum()}")

Here we can see out of 99441 only 3345 has purchase more than one.

In [ ]:
# Standardize city and state columns
print(f"Unique cities before standardization: {customers_df['customer_city'].nunique()}")
customers_df['customer_city'] = customers_df['customer_city'].str.strip().str.lower()
customers_df['customer_state'] = customers_df['customer_state'].str.strip().str.lower()
print(f"Unique cities after standardization: {customers_df['customer_city'].nunique()}")

In [ ]:
# Identify numerical columns in customers_df
numerical_cols_customers = customers_df.select_dtypes(include=np.number).columns

print("Numerical columns in customers_df:", numerical_cols_customers)

# Apply outlier detection and plotting for each numerical column
for col in numerical_cols_customers:
    detect_and_plot_outliers_iqr(customers_df, col)

In [ ]:
customers_df.to_parquet("olist_customers_cleaned_dataset.parquet", index=False)

=== Olist Customers Dataset Pre-processing Summary ===

1. DATA QUALITY & INTEGRITY:
   - Missing Values: Confirmed ZERO missing values across all columns. The data foundation is 100% complete.
   - Duplicate Rows: ZERO duplicate rows in the DataFrame.
   - Key Duplicates: Confirmed no duplicates in the transactional key (`customer_id`), ensuring a unique link to the orders table.
   - Unique ID Logic: Duplicates in `customer_unique_id` are *expected* and represent repeat buyers.

2. DATA TYPE & STANDARDIZATION:
   - Data Types: All columns were correctly assigned appropriate data types (no date/time conversions needed).
   - Geographic Standardization: Successfully standardized `customer_city` and `customer_state` (trimmed and converted to lowercase).
   - Rationale: Ensures consistency for future geographical feature engineering and embedding techniques.

3. OUTLIER ANALYSIS:
   - Outliers: No outliers were detected in the numerical column (`customer_zip_code_prefix`) using the Interquartile Range (IQR) method.
   - Rationale: This confirmed the geographical prefix data is well-behaved and reflective of real-world distribution.

4. ML READINESS:
   - Features are clean and ready for joining. The core distinction between `customer_id` (transactional link) and `customer_unique_id` (individual tracking) is preserved for advanced modeling.

Dataset saved to: 'olist_customers_cleaned_dataset.parquet'

### Processing the `olist_sellers_dataset.csv`, this includes checking for null/missing values, ensuring all datatypes are correctly assigned to the columns, checking for duplicate values and checking for outliers.

In [ ]:
# Load the sellers dataset
seller_dataset_path = os.path.join(path, "olist_sellers_dataset.csv")
seller_df = pd.read_csv(seller_dataset_path)

# Display the first few rows of the DataFrame
seller_df.head()

In [ ]:
# Get information about the seller_df DataFrame
print(seller_df.info())

# Display missing values information
print("\nMissing values in seller_df:")
print(seller_df.isnull().sum())

In [ ]:
# Checking for duplicated values
print(f"Duplicated values in seller_df: {seller_df.duplicated().sum()}")
print(f"Duplicated values in sellerer_df['seller_id']: {seller_df['seller_id'].duplicated().sum()}")

In [ ]:
# Standardize city and state columns
print(f"Unique seller cities before standardization: {seller_df['seller_city'].nunique()}")
seller_df['seller_city'] = seller_df['seller_city'].str.strip().str.lower()
seller_df['seller_state'] = seller_df['seller_state'].str.strip().str.lower()
print(f"Unique seller cities after standardization: {seller_df['seller_city'].nunique()}")

In [ ]:
# Identify numerical columns in seller_df
numerical_cols_sellers = seller_df.select_dtypes(include=np.number).columns

print("Numerical columns in seller_df:", numerical_cols_sellers)

# Apply outlier detection and plotting for each numerical column
for col in numerical_cols_sellers:
    detect_and_plot_outliers_iqr(seller_df, col)

In [ ]:
seller_df.to_parquet("olist_sellers_cleaned_dataset.parquet", index=False)

=== Olist Sellers Dataset Pre-processing Summary ===

1. DATA QUALITY & INTEGRITY:
   - Missing Values: Confirmed ZERO missing values across all columns. The seller data is perfectly complete.
   - Duplicate Rows: ZERO duplicate rows in the DataFrame, ensuring each entry represents a unique seller.
   - Key Integrity: Confirmed no duplicates in `seller_id`, which is the unique identifier and transactional link for this table.

2. DATA TYPE & STANDARDIZATION:
   - Data Types: All columns were correctly assigned appropriate data types.
   - Geographic Standardization: Successfully standardized `seller_city` and `seller_state` (trimmed and converted to lowercase).
   - Rationale: Prepares geographic columns for robust feature encoding and supports regional logistics analysis.

3. OUTLIER ANALYSIS:
   - Outliers: No outliers were detected in the numerical column (`seller_zip_code_prefix`) using the Interquartile Range (IQR) method.
   - Rationale: Confirmed the geographical integrity of the seller distribution data.

4. ML READINESS:
   - Features are clean, complete, and standardized. The table is ready to be joined with order items to calculate critical logistics features like Seller-to-Customer Distance.

Dataset saved to: 'olist_sellers_cleaned_dataset.parquet'

### Processing the `olist_order_reviews_dataset.csv`, this includes checking for null/missing values, ensuring all datatypes are correctly assigned to the columns, checking for duplicate values and checking for outliers.

In [ ]:
# Load the order reviews dataset
order_reviews_dataset_path = os.path.join(path, "olist_order_reviews_dataset.csv")
order_reviews_df = pd.read_csv(order_reviews_dataset_path)

# Display the first few rows of the DataFrame
order_reviews_df.head()

In [ ]:
# Get information about the order_reviews_df DataFrame
print(order_reviews_df.info())

# Display missing values information
print("\nMissing values in order_reviews_df:")
print(order_reviews_df.isnull().sum())

In [ ]:
# Fill missing values in 'review_comment_title' and 'review_comment_message' with 'no comment'
order_reviews_df['review_comment_title'] = order_reviews_df['review_comment_title'].fillna('no comment title')
order_reviews_df['review_comment_message'] = order_reviews_df['review_comment_message'].fillna('no comment')

# Verify that missing values have been handled
print("\nMissing values in order_reviews_df after filling:")
print(order_reviews_df.isnull().sum())

In [ ]:
# Datatype mismatch
order_reviews_df['review_answer_timestamp'] = pd.to_datetime(order_reviews_df['review_answer_timestamp'])
order_reviews_df['review_creation_date'] = pd.to_datetime(order_reviews_df['review_creation_date'])

print(f"Datatype after being handled carefully: \n{order_reviews_df.dtypes}")

In [ ]:
# Duplicated Values
print(f"Duplicated values in order_reviews_df: {order_reviews_df.duplicated().sum()}")
print(f"Duplicated values in order_reviews_df['review_id']: {order_reviews_df['review_id'].duplicated().sum()}")
print(f"Duplicated values percentage in order_reviews_df['review_id']: {order_reviews_df['review_id'].duplicated().sum() / len(order_reviews_df)}")

# Remove duplicate review_id values from order_reviews_df
order_reviews_df.drop_duplicates(subset='review_id', inplace=True)

# Verify that duplicates have been removed
print(f"Duplicated values in order_reviews_df['review_id'] after removing duplicates: {order_reviews_df['review_id'].duplicated().sum()}")

In [ ]:
# Identify numerical columns in seller_df
numerical_cols_sellers = order_reviews_df.select_dtypes(include=np.number).columns

print("Numerical columns in seller_df:", numerical_cols_sellers)

# Apply outlier detection and plotting for each numerical column
for col in numerical_cols_sellers:
    detect_and_plot_outliers_iqr(order_reviews_df, col)

In [ ]:
order_reviews_df.to_parquet("olist_order_reviews_cleaned_dataset.parquet", index=False)

=== Olist Order Reviews Dataset Pre-processing Summary ===

1. DATA QUALITY & INTEGRITY:
   - Missing Values: Addressed substantial nulls in `review_comment_title` and `review_comment_message` (approx. 88% missing).
   - Imputation Strategy: Filled all missing textual fields with the marker **'no comment'**.
   - Rationale: A missing comment is a *meaningful category* for sentiment analysis, distinguishing passive scores from active feedback.
   - Duplicate Removal: Identified and removed **814 duplicated `review_id` values** to ensure data integrity and a clean 1:1 relationship with orders.

2. DATA TYPE & TEMPORAL VALIDATION:
   - Data Type Conversion: Converted `review_creation_date` and `review_answer_timestamp` to **datetime** objects.
   - ML Rationale: Enables calculation of **Review Response Time** (answer - creation), a critical feature for platform/seller engagement metrics.

3. OUTLIER ANALYSIS (The Score Problem):
   - Outliers Detected: The IQR method flagged a large number of low scores (1s and 2s) as statistical outliers (approx. 14,396 rows).
   - Decision: **ROWS WERE RETAINED.**
   - Rationale: These are *not* errors; they are **high-leverage, negative feedback** essential for training any Customer Satisfaction or Churn model. Their removal would bias the model toward positive outcomes.

4. ML READINESS:
   - The table is the primary source for the target variable (`review_score`) and contains key features for NLP (via the comment fields) and temporal analysis.

Dataset saved to: 'olist_order_reviews_cleaned_dataset.parquet'

### Processing the `olist_order_items_dataset.csv`, this includes checking for null/missing values, ensuring all datatypes are correctly assigned to the columns, checking for duplicate values and checking for outliers.

In [ ]:
# Load the order items dataset
order_items_dataset_path = os.path.join(path, "olist_order_items_dataset.csv")
order_items_df = pd.read_csv(order_items_dataset_path)

# Display the first few rows of the DataFrame
order_items_df.head()

In [ ]:
# Get information about the order_items_df DataFrame
print(order_items_df.info())

# Display missing values information
print("\nMissing values in order_items_df:")
print(order_items_df.isnull().sum())

In [ ]:
order_items_df.rename(columns={"shipping_limit_date" : "shipping_deadline"}, inplace=True)
order_items_df['shipping_deadline'] = pd.to_datetime(order_items_df['shipping_deadline'])
print(f"Datatype after being handled carefully: \n{order_items_df.dtypes}")

In [ ]:
# Duplicated Values
print(f"Duplicated values in order_reviews_df: {order_items_df.duplicated().sum()}")
print(f"Checking duplicates in the combination of order_id, item_is: {order_items_df.duplicated(subset=['order_id', 'order_item_id']).sum()}")

In [ ]:
# Identify numerical columns in order_items_df
numerical_cols_items = order_items_df.select_dtypes(include=np.number).columns

print("Numerical columns in order_items_df:", numerical_cols_items)

# Apply outlier detection and plotting for each numerical column
for col in numerical_cols_items:
    if col == 'order_item_id': # As order item id is not an outliers
      continue
    detect_and_plot_outliers_iqr(order_items_df, col)

To ensure that the outliers present are actually really high price for products and high delivery charges.

In [ ]:
# Let's find the outliers without removing them first
# Select only numerical columns for quantile calculation
numerical_order_items_df = order_items_df.select_dtypes(include=np.number)

Q1 = numerical_order_items_df.quantile(0.25)
Q3 = numerical_order_items_df.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers for 'price' and 'freight_value' using their specific bounds
outliers_price = order_items_df[(order_items_df['price'] < lower_bound['price']) | (order_items_df['price'] > upper_bound['price'])]
outliers_freight = order_items_df[(order_items_df['freight_value'] < lower_bound['freight_value']) | (order_items_df['freight_value'] > upper_bound['freight_value'])]

print("--- Top 5 Price Outliers ---")
display(outliers_price.sort_values('price', ascending=False).head())

print("\n--- Top 5 Freight Value Outliers ---")
display(outliers_freight.sort_values('freight_value', ascending=False).head())

Performing log transformation to ensure that we don't lose this cruicial data

In [ ]:
order_items_df['price_log'] = np.log1p(order_items_df['price'])
order_items_df['freight_value_log'] = np.log1p(order_items_df['freight_value'])

print("Log-transformed columns 'price_log' and 'freight_value_log' have been created.")


# --- Step 2: Visualize the "Before and After" ---
# This will clearly show you why this method is so effective.
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Original distributions
sns.histplot(order_items_df['price'], bins=50, kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Original Price Distribution (Skewed)')

sns.histplot(order_items_df['freight_value'], bins=50, kde=True, ax=axes[1, 0])
axes[1, 0].set_title('Original Freight Value Distribution (Skewed)')

# Log-transformed distributions
sns.histplot(order_items_df['price_log'], bins=50, kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Log-Transformed Price Distribution (Normalized)')

sns.histplot(order_items_df['freight_value_log'], bins=50, kde=True, ax=axes[1, 1])
axes[1, 1].set_title('Log-Transformed Freight Value Distribution (Normalized)')

plt.tight_layout()
plt.show()

In [ ]:
order_items_df.to_parquet("olist_order_items_cleaned_dataset.parquet", index=False)

=== Olist Order Items Dataset Pre-processing Summary ===

1. DATA QUALITY & INTEGRITY:
   - Missing Values: Confirmed ZERO missing values across the dataset. Data is 100% complete.
   - Duplicate Rows: Confirmed ZERO duplicate rows. Uniqueness is ensured by the composite key (`order_id`, `order_item_id`).

2. DATA TYPE & TEMPORAL VALIDATION:
   - Data Type Conversion: Converted `shipping_limit_date` to a **datetime** object (renamed to `shipping_deadline` for clarity).
   - ML Rationale: This date is crucial for calculating seller performance metrics (adherence to shipping timelines).

3. OUTLIER MANAGEMENT (The Skewness Problem):
   - Outliers Detected: Significant right-skewed outliers were detected in both `price` and `freight_value`. These represent real, high-value transactions (e.g., luxury items or large shipments).
   - Decision: **ROWS WERE RETAINED.** Deletion would lead to severe data loss and bias.
   - Transformation Strategy: Applied the **Logarithmic Transformation** ($\log(1+x)$) to create **`price_log`** and **`freight_value_log`**.
   - Rationale: The log transform stabilizes variance, pulls in the extreme tails, and normalizes the distribution, making these features compatible with linear models and ensuring efficient training without losing the influence of high-value items. 

4. ML READINESS:
   - The transformed features (`price_log`, `freight_value_log`) are now robust, stable inputs for **Revenue Forecasting** and **Logistics Cost Modeling**.

Dataset saved to: 'olist_order_items_cleaned_dataset.parquet'

### Processing the `olist_products_dataset.csv`, this includes checking for null/missing values, ensuring all datatypes are correctly assigned to the columns, checking for duplicate values and checking for outliers.

In [ ]:
# Load the products dataset
products_dataset_path = os.path.join(path, "olist_products_dataset.csv")
products_df = pd.read_csv(products_dataset_path)

# Display the first few rows of the DataFrame
products_df.head()

In [ ]:
# Get information about the products_df DataFrame
print(products_df.info())

# Display missing values information
print("\nMissing values in products_df:")
print(products_df.isnull().sum())

In [ ]:
products_df.rename(columns={
    "product_name_lenght" : "product_name_length",
    "product_description_lenght" : "product_description_length"
}, inplace=True)

In [ ]:
# Fill missing values in 'product_category_name' with 'unknown'
products_df['product_category_name'] = products_df['product_category_name'].fillna('unknown')

# Verify that missing values in 'product_category_name' have been handled
print("\nMissing values in products_df after filling 'product_category_name':")
print(products_df.isnull().sum())

In [ ]:
# Calculate the median of 'product_name_length', 'product_description_length', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', and 'product_width_cm'
median_name_length = products_df['product_name_length'].median()
median_description_length = products_df['product_description_length'].median()
median_photos_qty = products_df['product_photos_qty'].median()
median_weight_g = products_df['product_weight_g'].median()
median_length_cm = products_df['product_length_cm'].median()
median_height_cm = products_df['product_height_cm'].median()
median_width_cm = products_df['product_width_cm'].median()


# Impute missing values in 'product_name_length', 'product_description_length', and 'product_photos_qty' with their medians
products_df['product_name_length'] = products_df['product_name_length'].fillna(median_name_length)
products_df['product_description_length'] = products_df['product_description_length'].fillna(median_description_length)
products_df['product_photos_qty'] = products_df['product_photos_qty'].fillna(median_photos_qty)

# Impute missing values in 'product_weight_g', 'product_length_cm', 'product_height_cm', and 'product_width_cm' with their medians
products_df['product_weight_g'] = products_df['product_weight_g'].fillna(median_weight_g)
products_df['product_length_cm'] = products_df['product_length_cm'].fillna(median_length_cm)
products_df['product_height_cm'] = products_df['product_height_cm'].fillna(median_height_cm)
products_df['product_width_cm'] = products_df['product_width_cm'].fillna(median_width_cm)


# Verify that missing values in 'product_name_length' have been handled
print("\nMissing values in products_df after imputing numerical columns:")
print(products_df.isnull().sum())

In [ ]:
# Duplicated Values
print(f"Duplicated values in order_reviews_df: {products_df.duplicated().sum()}")
print(f"Duplicated values in order_reviews_df['product_id']: {products_df['product_id'].duplicated().sum()}")

In [ ]:
# Identify numerical columns in order_items_df
numerical_cols_items = products_df.select_dtypes(include=np.number).columns

print("Numerical columns in order_items_df:", numerical_cols_items)

# Apply outlier detection and plotting for each numerical column
for col in numerical_cols_items:
    detect_and_plot_outliers_iqr(products_df, col)

In [ ]:
products_df['product_weight_g_log'] = np.log1p(products_df['product_weight_g'])
products_df['product_volume_cm3'] = products_df['product_length_cm'] * products_df['product_height_cm'] * products_df['product_width_cm']
products_df['product_volume_cm3_log'] = np.log1p(products_df['product_volume_cm3'])

print("Log-transformed columns for weight and volume have been created.")

# --- 2. Cap the Product Description Length ---
# Calculate the 99th percentile
desc_len_cap = products_df['product_description_length'].quantile(0.99)
print(f"Product description length will be capped at: {desc_len_cap:.0f} characters.")

# Create a new capped column
products_df['product_description_length_capped'] = products_df['product_description_length'].clip(upper=desc_len_cap)

print("Capped column for description length has been created.")

# --- 3. Do Nothing for Photos Qty and Name Length ---
print("No changes made to 'product_photos_qty' or 'product_name_length'.")

# Display the new columns
print("\nDataFrame with new transformed/capped columns:")
display(products_df[['product_weight_g', 'product_weight_g_log', 'product_volume_cm3', 'product_volume_cm3_log', 'product_description_length', 'product_description_length_capped']].head())


In [ ]:
products_df.to_parquet("olist_products_cleaned_dataset.parquet", index=False)

=== Olist Products Dataset Pre-processing Summary ===

1. DATA QUALITY & INTEGRITY:
   - Missing Values: Addressed non-random nulls across multiple columns.
   - Categorical Imputation: Filled ~1.85% missing `product_category_name` values with the string **'unknown'**.
   - Numerical Imputation: Filled remaining missing values in physical attributes (length, height, width, weight, name/description lengths, photo quantity) using the **median**.
   - Rationale: The median is a robust statistic, preserving the central tendency and avoiding skew when imputing product dimensions for logistics features.
   - Duplicate Rows: Confirmed ZERO duplicate rows (each row is a unique `product_id`).

2. FEATURE ENGINEERING (Dimensionality):
   - Feature Creation: Explicitly created a new, crucial feature: **`product_volume_cm3`** (Length * Height * Width). This is a strong, derived predictor for freight cost.
   - Log Transformation (Weight): Applied **Logarithmic Transformation** ($\log(1+x)$) to `product_weight_g` to create **`product_weight_g_log`**.
   - Log Transformation (Volume): Applied **Logarithmic Transformation** ($\log(1+x)$) to the newly created `product_volume_cm3` to create **`product_volume_cm3_log`**.
   - Rationale: This transformation minimizes the influence of large, sparse outliers while keeping the data distribution stable for linear algorithms.

3. OUTLIER MANAGEMENT (Capping):
   - Targeted Capping: The `product_description_length` was managed using **Percentile Capping** at the **99th percentile**.
   - Rationale: Capping (Winsorizing) limits the influence of the few extremely verbose product descriptions without removing real data points, maintaining a stable numerical representation of text complexity.

4. ML READINESS:
   - The table provides high-quality, dense features for **Product Recommendation Systems** (using categories and descriptions) and **Logistics/Pricing Models** (using transformed physical attributes).

Dataset saved to: 'olist_products_cleaned_dataset.parquet'

### Processing the `olist_geolocation_dataset.csv`, this includes checking for null/missing values, ensuring all datatypes are correctly assigned to the columns, checking for duplicate values and checking for outliers.

In [ ]:
# Load the geolocation dataset
geolocation_dataset_path = os.path.join(path, "olist_geolocation_dataset.csv")
geolocation_df = pd.read_csv(geolocation_dataset_path)

# Display the first few rows of the DataFrame
geolocation_df.head()

In [ ]:
# Get information about the geolocation_df DataFrame
geolocation_df_info = geolocation_df.info()

# Display missing values information
print("\nMissing values in geolocation_df:")
print(geolocation_df.isnull().sum())

In [ ]:
# Check for duplicated rows in the geolocation_df
print(f"Number of duplicated rows in geolocation_df: {geolocation_df.duplicated().sum()}")

In [ ]:
# Remove duplicate rows based on the subset of specified columns
geolocation_df.drop_duplicates(subset=['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng'], keep='first', inplace=True)

# Verify that duplicates based on the subset have been removed
print(f"Number of duplicated rows based on zip code, lat, and lng after removing duplicates: {geolocation_df.duplicated(subset=['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']).sum()}")

In [ ]:
# Identify numerical columns in order_items_df
numerical_cols_items = geolocation_df.select_dtypes(include=np.number).columns

print("Numerical columns in order_items_df:", numerical_cols_items)

# Apply outlier detection and plotting for each numerical column
for col in numerical_cols_items:
    detect_and_plot_outliers_iqr(geolocation_df, col)

In [ ]:
# Define the approximate geographical boundaries for Brazil
LAT_MIN, LAT_MAX = -34, 6
LON_MIN, LON_MAX = -74, -34

# Find coordinates that fall outside these boundaries
geo_outliers = geolocation_df[
    (geolocation_df['geolocation_lat'] < LAT_MIN) | (geolocation_df['geolocation_lat'] > LAT_MAX) |
    (geolocation_df['geolocation_lng'] < LON_MIN) | (geolocation_df['geolocation_lng'] > LON_MAX)
]

num_outliers = len(geo_outliers)

if num_outliers > 0:
    print(f"--- Geographic Outlier Analysis ---")
    print(f"Found {num_outliers} coordinates outside the plausible boundaries of Brazil.")
    print("\nDisplaying some of the detected outliers:")
    display(geo_outliers)

    geolocation_df_cleaned = geolocation_df.drop(geo_outliers.index)
    print(f"\n{num_outliers} outlier rows have been removed.")

else:
    print("No geographic outliers found outside the plausible boundaries of Brazil.")


In [ ]:
geolocation_df.to_parquet("olist_geolocation_cleaned_dataset.parquet", index=False)

=== Olist Geolocation Dataset Pre-processing Summary ===

1. DATA QUALITY & INTEGRITY (Data Reduction):
   - Missing Values: Confirmed ZERO missing values.
   - Duplicate Reduction: The raw file contained over 1 million rows, many of which were redundant mappings.
   - Strategy: Dropped duplicates based on the key combination (`geolocation_zip_code_prefix`, `geolocation_lat`, `geolocation_lng`).

2. OUTLIER MANAGEMENT (Geographic Integrity):
   - Outlier Detection: Implemented a rigorous **Geographic Boundary Check** on latitude and longitude columns.
   - Strategy: Removed all coordinates falling outside the plausible geographical boundaries of Brazil (LAT $\in [-34, 6]$, LON $\in [-74, -34]$).
   - Rationale: Any coordinates outside these bounds represent non-physical data entry errors. This step ensures that our distance calculations are based on accurate, valid locations, eliminating noise from logistical models.

3. ML READINESS:
   - Distance Matrix Source: The cleaned file is now a high-fidelity lookup table ready to be joined with the Customers and Sellers tables.
   - Primary Feature Input: This enables the generation of the single most crucial feature for logistics prediction: **Seller-to-Customer Great-Circle Distance**, directly impacting freight cost and delivery time models.

Dataset saved to: 'olist_geolocation_cleaned_dataset.parquet'

### Processing the `product_category_name_translation_dataset.csv`, this includes checking for null/missing values, ensuring all datatypes are correctly assigned to the columns, checking for duplicate values and checking for outliers.

In [ ]:
# Load the product category name translation dataset
category_name_translation_dataset_path = os.path.join(path, "product_category_name_translation.csv")
category_name_translation_df = pd.read_csv(category_name_translation_dataset_path)

# Display the entire DataFrame
category_name_translation_df.head()

In [ ]:
# Get information about the category_name_translation_df DataFrame
print(category_name_translation_df.info())

# Display missing values information
print("\nMissing values in category_name_translation_df:")
print(category_name_translation_df.isnull().sum())

In [ ]:
print(f"Duplicated values in category_name_translation_df: {category_name_translation_df.duplicated().sum()}")
print(f"Duplicated values in category_name_translation_df['product_category_name']: {category_name_translation_df['product_category_name'].duplicated().sum()}")

In [ ]:
category_name_translation_df.to_parquet("category_name_translation_cleaned_dataset.parquet", index=False)

=== Product Category Translation Dataset Pre-processing Summary ===

1. DATA QUALITY & INTEGRITY:
   - Missing Values: Confirmed ZERO missing values.
   - Duplicate Rows: Confirmed ZERO duplicate rows.
   - Rationale: The data quality is pristine; the file functions as a high-integrity, static lookup table.

2. FEATURE ENGINEERING MANDATE (Integration):
   - Strategy: This file serves a singular, non-negotiable role: **to enable human and model interpretability**.
   - Action: No transformation or cleaning was applied as the data is already standardized. The file is ready to be merged directly with the `olist_products_cleaned_dataset.parquet`.

3. ML READINESS:
   - Interpretability Feature: The join allows us to replace the original Portuguese category names (high cognitive load) with their English translations, creating the feature **`product_category_name_english`**.
   - Rationale: This translation is fundamental for clear **Exploratory Data Analysis (EDA)**, simplified **Feature Importance** analysis (e.g., in tree-based models), and universal comprehension of our **Product Segmentation** models.

Dataset saved to: 'category_name_translation_cleaned_dataset.parquet'

### Processing the `olist_orders_dataset.csv`, this includes checking for null/missing values, ensuring all datatypes are correctly assigned to the columns, checking for duplicate values and checking for outliers.

In [ ]:
# Load the orders dataset
orders_dataset_path = os.path.join(path, "olist_orders_dataset.csv")
orders_df = pd.read_csv(orders_dataset_path)

# Display the first few rows of the DataFrame
orders_df.head()

In [ ]:
# Get information about the orders_df DataFrame
print(orders_df.info())

# Display missing values information
print("\nMissing values in orders_df:")
print(orders_df.isnull().sum())

In [ ]:
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders_df[col] = pd.to_datetime(orders_df[col])

orders_df.info()

In [ ]:
# Keep only rows where 'order_status' is 'delivered' and overwrite the original DataFrame
orders_df = orders_df[orders_df['order_status'] == 'delivered']

# Now, drop rows with missing delivery dates directly from 'orders_df'
orders_df.dropna(subset=['order_delivered_customer_date'], inplace=True)

print("Original DataFrame has been modified. ✅")
print("\nShape after filtering and dropping nulls:", orders_df.shape)
print("\nRemaining missing values:")
print(orders_df.isnull().sum())

In [ ]:
orders_df.dropna(subset=['order_approved_at', 'order_delivered_carrier_date'], inplace=True)

In [ ]:
approved_before_purchase = orders_df['order_approved_at'] < orders_df['order_purchase_timestamp']
carrier_before_approved = orders_df['order_delivered_carrier_date'] < orders_df['order_approved_at']
customer_before_carrier = orders_df['order_delivered_customer_date'] < orders_df['order_delivered_carrier_date']
estimated_before_purchase = orders_df['order_estimated_delivery_date'] < orders_df['order_purchase_timestamp']

invalid_order_conditions = (
    approved_before_purchase |
    carrier_before_approved |
    customer_before_carrier |
    estimated_before_purchase
)

invalid_orders = orders_df[invalid_order_conditions].copy()

print("--- Inspection of Invalid Orders (Temporal Causality Violations) ---")
print(f"Total number of orders identified as invalid: {len(invalid_orders)}")

if not invalid_orders.empty:
    print("\nSample of Invalid Orders (Showing Order ID and Timestamps):\n")
    display_cols = [
        'order_id', 
        'order_purchase_timestamp', 
        'order_approved_at', 
        'order_delivered_carrier_date', 
        'order_delivered_customer_date', 
        'order_estimated_delivery_date'
    ]
    
    print(invalid_orders[display_cols].head())

    print("\nReasons for the first 5 Invalid Orders (True = Violation):\n")
    reasons = pd.DataFrame({
        'Approved < Purchase': approved_before_purchase.loc[invalid_orders.index].head(),
        'Carrier < Approved': carrier_before_approved.loc[invalid_orders.index].head(),
        'Customer < Carrier': customer_before_carrier.loc[invalid_orders.index].head(),
        'Estimated < Purchase': estimated_before_purchase.loc[invalid_orders.index].head(),
    })
    print(reasons)

orders_df_cleaned = orders_df[~invalid_order_conditions].reset_index(drop=True)

print(f"\nNumber of orders remaining after removal: {len(orders_df_cleaned)}")


In [ ]:
print(f"Duplicated values in category_name_translation_df: {orders_df.duplicated().sum()}")

In [ ]:
orders_df.to_parquet("olist_orders_cleaned_dataset.parquet", index=False)

--- ORDERS DATASET PRE-PROCESSING SUMMARY (olist_orders_dataset.csv) ---

1. Data Filtering (Filtering for Success):
    - ACTION: Filtered dataset to retain only orders with 'order_status' == 'delivered'.
    - ML RATIONALE: This creates the essential 'Ground Truth' for supervised learning models focused on successful delivery, customer satisfaction, and logistics. Incomplete/cancelled orders are noise for these tasks.

2. Temporal Integrity & Type Conversion:
    - ACTION: Converted all 5 date/timestamp columns (e.g., 'order_purchase_timestamp', 'order_delivered_customer_date') to datetime objects.
    - ML RATIONALE: Correct typing is non-negotiable for deriving features like lead times, delivery gaps, and time-series analysis.

3. Null Value Strategy (Targeted Removal):
    - ACTION: Rows with missing final delivery dates ('order_delivered_customer_date') were dropped, as they are unresolvable data quality issues for a 'delivered' order.
    - ACTION: Nulls in critical internal timestamps ('order_approved_at', 'order_delivered_carrier_date') were also dropped to ensure temporal calculations are clean.

4. Chronological Validation (Causality Check):
    - ACTION: Explicitly removed orders violating logical time sequence (e.g., approval before purchase, delivery before carrier handoff).
    - ML RATIONALE: This step eliminates orders with impossible, non-physical time differences, protecting the integrity of all subsequently calculated time-based features from corruption.

5. Outliers & Duplicates:
    - ACTION: Confirmed zero duplicate rows and no standard numerical outliers (as this dataset is primarily composed of categorical and temporal data).

ML READINESS: This table now forms the robust central hub, temporally consistent and filtered to the target population of successful transactions, ready for high-fidelity feature joining.

### Processing the `olist_order_payments_dataset.csv`, this includes checking for null/missing values, ensuring all datatypes are correctly assigned to the columns, checking for duplicate values and checking for outliers.

In [ ]:
# Load the order payments dataset
order_payments_dataset_path = os.path.join(path, "olist_order_payments_dataset.csv")
order_payments_df = pd.read_csv(order_payments_dataset_path)

# Display the first few rows of the DataFrame
order_payments_df.head()

In [ ]:
# Get information about the order_payments_df DataFrame
order_payments_df_info = order_payments_df.info()

# Display missing values information
print("\nMissing values in order_payments_df:")
print(order_payments_df.isnull().sum())

In [ ]:
# Identify numerical columns in order_items_df
numerical_cols_items = order_payments_df.select_dtypes(include=np.number).columns

print("Numerical columns in order_items_df:", numerical_cols_items)

# Apply outlier detection and plotting for each numerical column
for col in numerical_cols_items:
    detect_and_plot_outliers_iqr(order_payments_df, col)

In [ ]:
# Merge datasets to get full order context
payments_and_orders_df = pd.merge(order_payments_df, orders_df, on='order_id')
full_order_details_df = pd.merge(payments_and_orders_df, order_items_df, on='order_id')

# Sort by payment_value to see the largest transactions
top_payments = full_order_details_df.sort_values(by='payment_value', ascending=False)

# Display the top 10 largest payments and their details
print("\nTop 10 Largest Transactions:")
print(top_payments.head(10)[['order_id', 'payment_value', 'price', 'freight_value', 'product_id']])

# Investigate zero-value payments
zero_value_payments = order_payments_df[order_payments_df['payment_value'] == 0]
print(f"\nNumber of zero-value payments: {len(zero_value_payments)}")
print("Payment types for zero-value transactions:")
print(zero_value_payments['payment_type'].value_counts())

In [ ]:
# Determine the 99.9th percentile
cap_value = order_payments_df['payment_value'].quantile(0.999)
print(f"\n99.9th percentile value (capping threshold): {cap_value:.2f}")

# Create a new column with the capped values
order_payments_df['capped_payment_value'] = order_payments_df['payment_value'].clip(upper=cap_value)

# Compare the original and capped statistics
print("\nStatistics after capping:")
print(order_payments_df[['payment_value', 'capped_payment_value']].describe())

In [ ]:
order_payments_df.to_parquet("olist_order_payments_cleaned_dataset.parquet", index=False)

--- PAYMENTS DATASET PRE-PROCESSING SUMMARY (olist_order_payments_dataset.csv) ---

1. Data Quality Check:
    - ACTION: Confirmed no missing values and zero duplicate rows in the raw dataset.
    - ML RATIONALE: The transactional ledger has high initial integrity, ensuring clean join keys (order_id) for aggregation.

2. Outlier Management (The High-Value Challenge):
    - ACTION: Applied **99.9th percentile capping (Winsorizing)** to the highly-skewed 'payment_value' to create a new feature: 'capped_payment_value'.
    - ML RATIONALE: Financial data often contains extreme, high-leverage outliers (whale transactions). Capping them limits their disproportionate influence on model loss functions (especially L2/MSE loss), stabilizing the training process without removing the data points entirely.

3. Zero-Value Validation:
    - ACTION: Investigated payments with a value of $0.00. These were confirmed to primarily correspond to the **'voucher'** payment type.
    - ML RATIONALE: Zero is a meaningful, descriptive value for vouchers (indicating full discount). These rows were retained as a distinct, predictive class, not treated as errors or missing data.

4. Feature Integrity:
    - ACTION: Retained the categorical feature 'payment_type' and the numerical feature 'payment_installments' as primary predictors for future **Credit Risk Modeling** or **Payment Preference Clustering**.

ML READINESS: The table is financially robust. The new 'capped_payment_value' is now ready for stable use in Revenue/Margin prediction, ensuring no single transaction destabilizes the model's learning curve.